In [3]:
import pandas as pd

## 등록된 제품 수

In [4]:
# 등록된 제품 수 by category, date
read_path = '../csv/products.csv'
df_product = pd.read_csv(read_path)

df_product.tail()

# status : 0: 판매중, 1: 예약됨, 2: 삭제, 3: 판매완료

,on_at,close_at,category_id,status,p_biz,products
12460795,2017-11-15,2018-01-19,310030020.0,2,0.0,1
12460796,2018-12-09,2018-12-11,750200300.0,3,0.0,1
12460797,2018-07-10,2018-07-23,310040050.0,2,0.0,1
12460798,2018-09-09,2019-03-01,999.0,3,0.0,1
12460799,2017-05-18,2017-06-24,320040500.0,3,0.0,1


In [5]:
# 카테고리별 등록된 상품

df_on = df_product.drop(['close_at', 'status'], axis=1).rename(columns={'on_at': 'date_at'})

df_on['month_at'] = df_on['date_at'].str[:7] # day -> month
df_on['category_3'] = df_on['category_id'].astype(str).str[:3] # 1 depth category
df_on['category_6'] = df_on['category_id'].astype(str).str[:6] # 2 depth category

df_on_month = df_on.groupby(['month_at', 'p_biz', 'category_3', 'category_6'], as_index=False).agg({'products': 'sum'})

df_on_month = df_on_month.rename(columns={'products': 'enrolled'})

df_on_month.tail()

,month_at,p_biz,category_3,category_6,enrolled
9617,2019-05,1.0,900,900320,5
9618,2019-05,1.0,900,900330,1
9619,2019-05,1.0,900,900400,2
9620,2019-05,1.0,910,910200,3
9621,2019-05,1.0,999,999.0,3


In [6]:
# save
save_path = '../csv/tmp/df_on_month.csv'
df_on_month.to_csv(save_path, index=False, mode='w', header=True)

In [7]:
# 1-Depth Category

In [8]:
df_on_month_3 = df_on_month.groupby(['month_at', 'category_3', 'p_biz'], as_index=False).agg({'enrolled': 'sum'})

df_on_month_3 = df_on_month_3.rename(columns={'category_3': 'category'})

df_on_month_3.tail()

,month_at,category,p_biz,enrolled
1054,2019-05,900,1.0,9
1055,2019-05,910,0.0,40
1056,2019-05,910,1.0,3
1057,2019-05,999,0.0,32
1058,2019-05,999,1.0,3


In [9]:
# 2-Depth Category

In [10]:
df_on_month_6 = df_on_month.groupby(['month_at', 'category_6', 'p_biz'], as_index=False).agg({'enrolled': 'sum'})

df_on_month_6 = df_on_month_6.rename(columns={'category_6': 'category'})

df_on_month_6.tail()

,month_at,category,p_biz,enrolled
9617,2019-05,910200,1.0,3
9618,2019-05,910400,0.0,2
9619,2019-05,910800,0.0,1
9620,2019-05,999.0,0.0,32
9621,2019-05,999.0,1.0,3


## 카테고리 파일 읽기

In [11]:
# 카테고리

path = '../csv/categories.csv'

df_category = pd.read_csv(path)

df_category['category'] = df_category['category'].astype(str)

df_category = df_category[['category', 'name']]

df_category.tail()

,category,name
1219,240100030,서비스/미디어
1220,240100050,생산/기능직
1221,240100070,상담영업
1222,240100090,기타 업종
1223,240100110,단기알바


## 번프 완료 수

In [47]:
read_path = '../csv/bunp_category_since_1801.csv'
df_bunp = pd.read_csv(read_path)

df_bunp['category_3'] = df_bunp['category'].astype(str).str[:3] # 1 depth category
df_bunp['category_6'] = df_bunp['category'].astype(str).str[:6] # 2 depth category

df_bunp.tail()

,category,name,updated_at,p_biz,selled_count,bunp_amount,category_3,category_6
491586,320060999.0,기타(가디건),2019-04-30,1,1,38000,320,320060
491587,500111006.0,스커트/치마,2019-04-30,0,1,23500,500,500111
491588,700800200.0,남성 수영복/레쉬가드,2019-04-25,0,1,30000,700,700800
491589,400070400.0,왕골/바캉스모자,2019-04-28,0,1,12000,400,400070
491590,400070400.0,왕골/바캉스모자,2019-04-30,1,1,8000,400,400070


In [48]:
# 1-Depth Category

In [49]:
df_bunp_cate_3 = pd.merge(df_bunp, df_category[['category', 'name']], 
                        how='left', left_on=['category_3'], right_on=['category'])

df_bunp_cate_3['month_at'] = df_bunp_cate_3['updated_at'].str[:7]

df_bunp_cate_3 = df_bunp_cate_3.rename(columns={'category_y': 'category', 'name_y': 'category_name'})

df_bunp_cate_3 = df_bunp_cate_3.groupby(['month_at', 'p_biz', 'category',
                                     'category_name'], as_index=False).agg({'bunp_amount': 'sum',
                                                           'selled_count': 'sum'})

df_bunp_cate_3.tail()

,month_at,p_biz,category,category_name,bunp_amount,selled_count
559,2019-04,1,750,차량/오토바이,69398397,612
560,2019-04,1,800,생활/문구/가구/식품,228936349,8651
561,2019-04,1,900,도서/티켓/취미/애완,228329623,7202
562,2019-04,1,910,스타굿즈,34829826,1654
563,2019-04,1,999,기타,60837842,1604


In [50]:
# 2-Depth Category

In [51]:
df_bunp_cate_6 = pd.merge(df_bunp, df_category[['category', 'name']], 
                        how='left', left_on=['category_6'], right_on=['category'])

df_bunp_cate_6['month_at'] = df_bunp_cate_6['updated_at'].str[:7]

df_bunp_cate_6 = df_bunp_cate_6.rename(columns={'category_y': 'category', 'name_y': 'category_name'})

df_bunp_cate_6 = df_bunp_cate_6.groupby(['month_at', 'p_biz', 'category',
                                     'category_name'], as_index=False).agg({'bunp_amount': 'sum',
                                                           'selled_count': 'sum'})

df_bunp_cate_6.tail()

,month_at,p_biz,category,category_name,bunp_amount,selled_count
4323,2019-04,1,910400,솔로(여),363900,20
4324,2019-04,1,910500,솔로(남),117000,8
4325,2019-04,1,910600,배우(남),115400,14
4326,2019-04,1,910700,배우(여),7000,1
4327,2019-04,1,910800,기타(방송인),540900,24


## 번프와 등록된 상품 합치기

In [52]:
# 1-Depth Category

In [53]:
df_bunp_enroll_3 = pd.merge(df_bunp_cate_3, df_on_month_3, on=['month_at', 'category', 'p_biz'])

# save
save_path = '../csv/tmp/df_bunp_enroll_3.csv'
df_bunp_enroll_3.to_csv(save_path, index=False, mode='w', header=True)

df_bunp_enroll_3.tail()

,month_at,p_biz,category,category_name,bunp_amount,selled_count,enrolled
556,2019-04,1,750,차량/오토바이,69398397,612,42746
557,2019-04,1,800,생활/문구/가구/식품,228936349,8651,17933
558,2019-04,1,900,도서/티켓/취미/애완,228329623,7202,22358
559,2019-04,1,910,스타굿즈,34829826,1654,3551
560,2019-04,1,999,기타,60837842,1604,8434


In [54]:
# 2-Depth Category

In [55]:
df_bunp_enroll_6 = pd.merge(df_bunp_cate_6, df_on_month_6, on=['month_at', 'category', 'p_biz'])

# save
save_path = '../csv/tmp/df_bunp_enroll_6.csv'
df_bunp_enroll_6.to_csv(save_path, index=False, mode='w', header=True)

df_bunp_enroll_6.tail()

,month_at,p_biz,category,category_name,bunp_amount,selled_count,enrolled
4316,2019-04,1,910400,솔로(여),363900,20,10
4317,2019-04,1,910500,솔로(남),117000,8,16
4318,2019-04,1,910600,배우(남),115400,14,3
4319,2019-04,1,910700,배우(여),7000,1,4
4320,2019-04,1,910800,기타(방송인),540900,24,30


## 클릭 수

In [38]:
read_path = '../csv/clicked.csv'
df_click = pd.read_csv(read_path)


df_click['category_3'] = df_click['category_id'].astype(str).str[:3] # 1 depth category
df_click['category_6'] = df_click['category_id'].astype(str).str[:6] # 2 depth category

df_click.tail()

,date_at,category_id,name,p_biz,clicked,category_3,category_6
966683,2019-04-27,320120600.0,NaN,1.0,1,320,320120
966684,2019-04-28,240090030.0,프로그래머,0.0,1,240,240090
966685,2019-04-28,410200.0,썬케어,1.0,1,410,410200
966686,2019-04-30,240100050.0,생산/기능직,0.0,1,240,240100
966687,2019-04-30,900230.0,상품권,0.0,1,900,900230


In [39]:
# 1-Depth Category

In [40]:
df_click_cate_3 = pd.merge(df_click, df_category[['category', 'name']], 
                        how='left', left_on=['category_3'], right_on=['category'])

df_click_cate_3['month_at'] = df_click_cate_3['date_at'].str[:7]

df_click_cate_3 = df_click_cate_3.rename(columns={'category_y': 'category', 'name_y': 'category_name'})

df_click_cate_3 = df_click_cate_3.groupby(['month_at', 'p_biz', 'category',
                                     'category_name'], as_index=False).agg({'clicked': 'sum'})

df_click_cate_3.tail()

,month_at,p_biz,category,category_name,clicked
603,2019-04,1.0,750,차량/오토바이,4747084
604,2019-04,1.0,800,생활/문구/가구/식품,3914021
605,2019-04,1.0,900,도서/티켓/취미/애완,3514745
606,2019-04,1.0,910,스타굿즈,823679
607,2019-04,1.0,999,기타,1382772


In [41]:
# 2-Depth Category

In [42]:
df_click_cate_6 = pd.merge(df_click, df_category[['category', 'name']], 
                        how='left', left_on=['category_6'], right_on=['category'])

df_click_cate_6['month_at'] = df_click_cate_6['date_at'].str[:7]

df_click_cate_6 = df_click_cate_6.rename(columns={'category_y': 'category', 'name_y': 'category_name'})

df_click_cate_6 = df_click_cate_6.groupby(['month_at', 'p_biz', 'category',
                                     'category_name'], as_index=False).agg({'clicked': 'sum'})

df_click_cate_6.tail()

,month_at,p_biz,category,category_name,clicked
5654,2019-04,1.0,910400,솔로(여),4046
5655,2019-04,1.0,910500,솔로(남),4854
5656,2019-04,1.0,910600,배우(남),5306
5657,2019-04,1.0,910700,배우(여),911
5658,2019-04,1.0,910800,기타(방송인),19232


## 클릭 수와 번프, 등록된 상품 합치기

In [56]:
# 1-Depth Category

In [57]:
df_bunp_enroll_click_3 = pd.merge(df_click_cate_3[['month_at', 'category', 'p_biz','clicked']], df_bunp_enroll_3,
                                  on=['month_at', 'category', 'p_biz'])

# save
save_path = '../csv/tmp/df_bunp_enroll_click_3.csv'
df_bunp_enroll_click_3.to_csv(save_path, index=False, mode='w', header=True)

df_bunp_enroll_click_3.tail()

,month_at,category,p_biz,clicked,category_name,bunp_amount,selled_count,enrolled
556,2019-04,750,1.0,4747084,차량/오토바이,69398397,612,42746
557,2019-04,800,1.0,3914021,생활/문구/가구/식품,228936349,8651,17933
558,2019-04,900,1.0,3514745,도서/티켓/취미/애완,228329623,7202,22358
559,2019-04,910,1.0,823679,스타굿즈,34829826,1654,3551
560,2019-04,999,1.0,1382772,기타,60837842,1604,8434


In [58]:
# 2-Depth Category

In [59]:
df_bunp_enroll_click_6= pd.merge(df_click_cate_6[['month_at', 'category', 'clicked', 'p_biz']], df_bunp_enroll_6,
                                  on=['month_at', 'category', 'p_biz'])

# save
save_path = '../csv/tmp/df_bunp_enroll_click_6.csv'
df_bunp_enroll_click_6.to_csv(save_path, index=False, mode='w', header=True)

df_bunp_enroll_click_6.tail()

,month_at,category,clicked,p_biz,category_name,bunp_amount,selled_count,enrolled
4316,2019-04,910400,4046,1.0,솔로(여),363900,20,10
4317,2019-04,910500,4854,1.0,솔로(남),117000,8,16
4318,2019-04,910600,5306,1.0,배우(남),115400,14,3
4319,2019-04,910700,911,1.0,배우(여),7000,1,4
4320,2019-04,910800,19232,1.0,기타(방송인),540900,24,30


## 키워드 검색 - 보류

In [13]:
# 키워드 검색 수 by date
read_path = '../csv/keywords_1901_1904.csv'
df_keyword_1901_1904 = pd.read_csv(read_path)

read_path = '../csv/keywords_1807_1812.csv'
df_keyword_1807_1812 = pd.read_csv(read_path)

df_keyword = pd.concat([df_keyword_1901_1904, df_keyword_1807_1812])

# save
save_path = '../csv/tmp/df_keyword.csv'
df_keyword.to_csv(save_path, index=False, mode='w', header=True)

df_keyword.tail()

,date_at,keyword,search_num
81942714,2018-12-31,ìœ„ì¸ì „,1
81942715,2018-12-31,실리콘케이스 스카이블루,1
81942716,2018-12-31,오프화이트 갤럭시 후드집업,1
81942717,2018-12-31,후방카메라cctv,2
81942718,2018-12-31,패딩오버트라우져,1


In [4]:
read_path = '../csv/tmp/df_keyword.csv'
df_keyword = pd.read_csv(read_path)

df_keyword.tail()

,date_at,keyword,search_num
141683920,2018-12-31,ìœ„ì¸ì „,1
141683921,2018-12-31,실리콘케이스 스카이블루,1
141683922,2018-12-31,오프화이트 갤럭시 후드집업,1
141683923,2018-12-31,후방카메라cctv,2
141683924,2018-12-31,패딩오버트라우져,1


In [5]:
read_path = '../csv/keyword_category_detail.csv'
df_category = pd.read_csv(read_path).drop(['_count', '_num'], axis=1)

df_category = df_category.rename(columns={'name': 'category_name'})

df_category.tail()


,keyword,category,category_name
1891622,💖신상🌸,320030100.0,맨투맨 티셔츠
1891623,💬찢청,320130200.0,청/데님 반바지
1891624,💯발렌시아가 빌 탑,400010100.0,숄더백
1891625,🚛정품 MCM,400083200.0,남성 벨트
1891626,🤘구찌 지갑,400060400.0,남자 중/반지갑


In [ ]:
df_key_cate = pd.merge(df_keyword, df_category[['keyword', 'category']],
                       how = 'left', on = ['keyword'])

df_key_cate.tail()